In [2]:
from config import settings
from src.partition import Partition
from src.generators import PartitionClass
from collections import Counter
import numpy as np
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

from keras.models import Sequential, Model, load_model
from keras.layers import (
    Dense, Conv2D, Flatten, 
    ConvLSTM2D, BatchNormalization,
    Conv3D, Input, MaxPooling2D, UpSampling2D,
)

from keras.callbacks.callbacks import EarlyStopping, ReduceLROnPlateau

Using TensorFlow backend.


# Keras Stability Model

In [83]:
test = Counter(p._next_oblak_step[1] for p in PartitionClass(n).grid_group(n))

In [84]:
test

Counter({(1,): 301,
         (): 378,
         (2,): 276,
         (3,): 251,
         (3, 1): 191,
         (4,): 226,
         (4, 1): 171,
         (5,): 201,
         (1, 1): 254,
         (5, 1): 151,
         (4, 2): 153,
         (6,): 176,
         (6, 1): 131,
         (5, 2): 134,
         (7,): 151,
         (1, 1, 1): 193,
         (2, 1): 229,
         (7, 1): 111,
         (6, 2): 115,
         (5, 3): 118,
         (8,): 126,
         (5, 3, 1): 79,
         (8, 1): 91,
         (7, 2): 96,
         (6, 3): 100,
         (9,): 101,
         (1, 1, 1, 1): 142,
         (2, 1, 1): 170,
         (6, 3, 1): 66,
         (9, 1): 71,
         (2, 2): 204,
         (8, 2): 77,
         (7, 3): 82,
         (6, 4): 86,
         (10,): 76,
         (3, 1, 1): 154,
         (7, 3, 1): 53,
         (6, 4, 1): 55,
         (10, 1): 51,
         (9, 2): 58,
         (8, 3): 64,
         (7, 4): 69,
         (11,): 51,
         (1, 1, 1, 1, 1): 101,
         (2, 1, 1, 1): 121,
       

In [85]:
[p for p in PartitionClass(n).grid_group(n) if p._next_oblak_step[1] == Partition(19, 6, 1)]

[(19, 6, 1)]

In [4]:
n = 28

In [75]:
training_partitions = [p for p in PartitionClass(n).grid_group(n) if p.is_stable]
training_matrices = [p.fit_matrix(n) for p in training_partitions]
training_targets = [p._next_oblak_step[1].fit_matrix(n) for p in training_partitions]

X = np.array(training_matrices)
y = np.array(training_targets)

X_train, X_test, y_train, y_test = train_test_split(X, y)

X_train = X_train.reshape(len(X_train), n, n, 1)
X_test = X_test.reshape(len(X_test), n, n, 1)
y_train = y_train.reshape(len(y_train), n, n, 1)
y_test = y_test.reshape(len(y_test), n, n, 1)

# reverse training
X_train, y_train = y_train, X_train
X_test, y_test = y_test, X_test

In [76]:
input_img = Input(shape=(n, n, 1))

x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (4, 4, 8) i.e. 128-dimensional

x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

model = Model(input_img, decoded)
model.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 14, 14, 8)         1160      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 7, 7, 8)           0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 7, 7, 8)           584       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 4, 4, 8)           0   

In [77]:
lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
es = EarlyStopping(monitor='val_loss', min_delta=.001, patience=50, verbose=0, mode='auto', baseline=None, restore_best_weights=False)
cbs = [es, lr]

In [78]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=1000, batch_size=10, workers = 4)

Train on 504 samples, validate on 169 samples
Epoch 1/1000
504/504 [==============================] - 1s 3ms/step - loss: 0.2403 - accuracy: 0.9528 - val_loss: 0.0476 - val_accuracy: 0.9704
Epoch 2/1000
504/504 [==============================] - 1s 1ms/step - loss: 0.0329 - accuracy: 0.9812 - val_loss: 0.0183 - val_accuracy: 0.9934
Epoch 3/1000
504/504 [==============================] - 1s 2ms/step - loss: 0.0167 - accuracy: 0.9929 - val_loss: 0.0189 - val_accuracy: 0.9913
Epoch 4/1000
504/504 [==============================] - 1s 2ms/step - loss: 0.0128 - accuracy: 0.9946 - val_loss: 0.0128 - val_accuracy: 0.9943
Epoch 5/1000
504/504 [==============================] - 1s 2ms/step - loss: 0.0107 - accuracy: 0.9955 - val_loss: 0.0087 - val_accuracy: 0.9965
Epoch 6/1000
504/504 [==============================] - 1s 2ms/step - loss: 0.0098 - accuracy: 0.9957 - val_loss: 0.0068 - val_accuracy: 0.9974
Epoch 7/1000
504/504 [==============================] - 1s 1ms/step - loss: 0.0090 - accur

KeyboardInterrupt: 

# Validation (Micro)


In [79]:
# 100% accuracy
print(f'Accuracy: {model.evaluate(X_test, y_test, verbose = False)[1] * 100}%')
print(f'AUC Score: {roc_auc_score(y_test.flatten(), model.predict(X_test).flatten()) * 100}%')

Accuracy: 99.73583817481995%
AUC Score: 99.99326035867931%


In [10]:
rand_index = np.random.randint(len(X_test))
random_input = X_test[rand_index]
actual_original = y_test[rand_index]

model_output = model.predict(random_input.reshape(1, n, n, 1)).reshape(n, n).round()

original_ptn = []
for row in actual_original:
    if row.sum():
        original_ptn.append(int(row.sum()))

oblak_ptn = []
for row in random_input:
    if row.sum():
        oblak_ptn.append(int(row.sum()))

output_ptn = []
for row in model_output:
    if row.sum():
        output_ptn.append(int(row.sum())) 
    
original_ptn = Partition(original_ptn)
oblak_ptn = Partition(oblak_ptn)
predicted_original = Partition(output_ptn)


print('OBLAK')
print(oblak_ptn)
print()

print('ORIGINAL')
print(original_ptn)
print()

print('PREDICTED')
print(predicted_original)

NameError: name 'X_test' is not defined

# Validation (Macro)

In [61]:
def macro_validate_partition(ptn_matrix, show = False):
    
    model_output = model.predict(ptn_matrix.reshape(1, n, n, 1)).reshape(n, n).round()

    input_ptn = []
    for row in ptn_matrix:
        if row.sum():
            input_ptn.append(int(row.sum()))

    output_ptn = []
    for row in model_output:
        if row.sum():
            output_ptn.append(int(row.sum())) 

    original_ptn = Partition(input_ptn)
    predicted_ptn = Partition(output_ptn)

    next_ptn = original_ptn._next_oblak_step[1]
    
    if show:
        
        print('ORIGINAL')
        print(original_ptn)
        print()

        print('NEXT')
        print(next_ptn._next_oblak_step[1])
        print()

        print('PREDICTED')
        print(predicted_ptn)
        
        
    return next_ptn == predicted_ptn

In [63]:
macro_validate_partition(Partition(28).fit_matrix(28), show = True)

ORIGINAL
▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢

NEXT
▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢

PREDICTED
▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢
▢▢▢
▢▢▢
▢▢
▢
▢
▢
▢
▢
▢


False

In [64]:
rights = []
wrongs = []
for matrix in X_test:
    if macro_validate_partition(matrix):
        rights.append(matrix)
        
    else:
        wrongs.append(matrix)
        
print(f'Macro Accuracy: {len(rights) / (len(wrongs) + len(rights)) * 100:.2f}%')

Macro Accuracy: 0.00%


In [65]:
len(rights) / (len(wrongs) + len(rights))

0.0